In [ ]:
!git clone https://github.com/pliang279/MultiBench.git
%cd MultiBench
!pip install gdown && gdown https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU
!pip install memory_profiler

Cloning into 'MultiBench'...
remote: Enumerating objects: 6943, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 6943 (delta 72), reused 121 (delta 60), pack-reused 6789
Receiving objects: 100% (6943/6943), 51.07 MiB | 20.91 MiB/s, done.
Resolving deltas: 100% (4258/4258), done.
/content/MultiBench
Downloading...
From (original): https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU
From (redirected): https://drive.google.com/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU&confirm=t&uuid=13024971-1adc-4598-96d3-2f958c5a4f18
To: /content/MultiBench/mosi_raw.pkl
100% 357M/357M [00:04<00:00, 76.6MB/s]


Downloading...
From (original): https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU
From (redirected): https://drive.google.com/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU&confirm=t&uuid=bd959679-f18c-4922-b1b0-6df26ec1e384
To: /content/MultiBench/mosi_raw.pkl
100% 357M/357M [00:04<00:00, 78.6MB/s]


In [ ]:
import torch
import sys
import os
sys.path.append(os.getcwd())
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from unimodals.common_models import GRU, MLP, Sequential, Identity  # noqa
from training_structures.Supervised_Learning import train, test  # noqa
from datasets.affect.get_data import get_dataloader  # noqa
from fusions.common_fusions import ConcatEarly  # noqa


# # mosi_data.pkl, mosei_senti_data.pkl
# # mosi_raw.pkl, mosei_senti_data.pkl, sarcasm.pkl, humor.pkl
# # raw_path: mosi.hdf5, mosei.hdf5, sarcasm_raw_text.pkl, humor_raw_text.pkl
# # traindata, validdata, testdata = get_dataloader('/home/pliang/multibench/affect/pack/mosi/mosi_raw.pkl', robust_test=False)
traindata, validdata, testdata = get_dataloader(
    '/content/MultiBench/mosi_raw.pkl', robust_test=False, max_pad=True, data_type='mosi', max_seq_len=50)

# # mosi/mosei
# encoders = [Identity().cuda(), Identity().cuda(), Identity().cuda()]
# head = Sequential(GRU(409, 512, dropout=True, has_padding=False,
#                   batch_first=True, last_only=True), MLP(512, 512, 1))


# # humor/sarcasm
# # encoders = [Identity().cuda(),Identity().cuda(),Identity().cuda()]
# # head = Sequential(GRU(752, 1128, dropout=True, has_padding=False, batch_first=True, last_only=True), MLP(1128, 512, 1)).cuda()

# fusion = ConcatEarly().cuda()

# train(encoders, fusion, head, traindata, validdata, 100, task="regression", optimtype=torch.optim.AdamW,
#       is_packed=False, lr=1e-3, save='mosi_ef_r0.pt', weight_decay=0.01, objective=torch.nn.L1Loss())

# print("Testing:")
# model = torch.load('/content/mosi_ef_r0.pt').cuda()
model = torch.load('/content/mosi_ef_r0.pt')
test(model, testdata, 'affect', is_packed=False,
     criterion=torch.nn.L1Loss(), task="posneg-classification", no_robust=True)

acc: 0.6112804878048781, 0.6020408163265306
Inference Time: 3.120678186416626
Inference Params: 1680897


In [ ]:
dir(traindata)

In [ ]:
type(traindata)

torch.utils.data.dataloader.DataLoader

In [ ]:
type(traindata.dataset)

datasets.affect.get_data.Affectdataset

In [ ]:
type(traindata.dataset.dataset)

dict

In [ ]:
traindata.dataset.dataset.keys()

dict_keys(['vision', 'audio', 'text', 'labels', 'id'])

In [ ]:
traindata.dataset.dataset['vision'].shape, traindata.dataset.dataset['audio'].shape, traindata.dataset.dataset['text'].shape

((1283, 50, 35), (1283, 50, 74), (1283, 50, 300))

In [ ]:
import torch
import sys
import os
sys.path.append(os.getcwd())
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from unimodals.common_models import GRU, MLP, Sequential, Identity  # noqa
from training_structures.unimodal import train, test  # noqa
from datasets.affect.get_data import get_dataloader  # noqa
from fusions.common_fusions import ConcatEarly  # noqa

# mosi_data.pkl, mosei_senti_data.pkl
# mosi_raw.pkl, mosei_raw.pkl, sarcasm.pkl, humor.pkl
# traindata, validdata, testdata = get_dataloader('/home/pliang/multibench/affect/pack/mosi/mosi_raw.pkl', robust_test=False)
traindata, validdata, testdata = get_dataloader(
    '/content/MultiBench/mosi_raw.pkl', robust_test=False, max_pad=True, data_type='mosi', max_seq_len=50)

modality_num = 2

# mosi/mosei
encoder = GRU(300, 600, dropout=True, has_padding=False,
              batch_first=True, last_only=True)
head = MLP(600, 512, 1)


train(encoder, head, traindata, validdata, 200, task="regression", optimtype=torch.optim.AdamW, lr=2e-3,
      weight_decay=0.01, criterion=torch.nn.L1Loss(), save_encoder='encoder_modality2.pt', save_head='head_modality2.pt', modalnum=modality_num)



In [ ]:
print("Testing when modality set to :2")
encoder = torch.load('encoder.pt')
head = torch.load('head.pt')
test(encoder, head, testdata, 'affect', criterion=torch.nn.L1Loss(),
     task="posneg-classification", modalnum=modality_num, no_robust=True)

Testing:
loss: tensor(1.4386)
acc: 0.4329268292682927, 0.4139941690962099
Inference Time: 2.0616798400878906
Inference Params: 1931825


In [ ]:
import torch
import sys
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
# sys.path.append(os.getcwd())
# sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

device(type='cuda')

In [ ]:
from unimodals.common_models import GRU, MLP, Sequential, Identity  # noqa
from training_structures.unimodal import train, test  # noqa
from datasets.affect.get_data import get_dataloader  # noqa
from fusions.common_fusions import ConcatEarly  # noqa

# mosi_data.pkl, mosei_senti_data.pkl
# mosi_raw.pkl, mosei_raw.pkl, sarcasm.pkl, humor.pkl
# traindata, validdata, testdata = get_dataloader('/home/pliang/multibench/affect/pack/mosi/mosi_raw.pkl', robust_test=False)
traindata, validdata, testdata = get_dataloader(
    '/content/MultiBench/mosi_raw.pkl', robust_test=False, max_pad=True, data_type='mosi', max_seq_len=50)

modality_num = 2

# mosi/mosei

encoder = GRU(300, 600, dropout=True, has_padding=False,
              batch_first=True, last_only=True).cuda()
head = MLP(600, 512, 1).cuda()


train(encoder, head, traindata, validdata, 200, task="regression", optimtype=torch.optim.AdamW, lr=2e-3,
      weight_decay=0.01, criterion=torch.nn.L1Loss(), save_encoder='encoder.pt', save_head='head.pt', modalnum=modality_num)

print("Testing:")
encoder = torch.load('encoder.pt').cuda()
head = torch.load('head.pt')
test(encoder, head, testdata, 'affect', criterion=torch.nn.L1Loss(),
     task="posneg-classification", modalnum=modality_num, no_robust=True)

Epoch 0 train loss: tensor(1.3502, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 0 valid loss: tensor(1.3827, device='cuda:0')
Saving Best
Epoch 1 train loss: tensor(1.3311, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 1 valid loss: tensor(1.3888, device='cuda:0')
Epoch 2 train loss: tensor(1.3195, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2 valid loss: tensor(1.3815, device='cuda:0')
Saving Best
Epoch 3 train loss: tensor(1.3183, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 3 valid loss: tensor(1.3866, device='cuda:0')
Epoch 4 train loss: tensor(1.3192, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 4 valid loss: tensor(1.3824, device='cuda:0')
Epoch 5 train loss: tensor(1.3150, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 5 valid loss: tensor(1.3844, device='cuda:0')
Epoch 6 train loss: tensor(1.3177, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 6 valid loss: tensor(1.3855, device='cuda:0')
Epoch 7 train loss: tensor(1.3185, device='cuda:0', grad_fn=<DivBackward0>)
Epoc

# Results
1. Only vision
2. Only audio
3. Only text
4. All three modalities

In [ ]:
encoder = GRU(35, 600, dropout=True, has_padding=False,
              batch_first=True, last_only=True).cuda()
head = MLP(600, 512, 1).cuda()

train(encoder, head, traindata, validdata, 200, task="regression", optimtype=torch.optim.AdamW, lr=2e-3,
      weight_decay=0.01, criterion=torch.nn.L1Loss(), save_encoder='encoder0.pt', save_head='head0.pt', modalnum= 0)


print("Testing:")
encoder = torch.load('encoder0.pt').cuda()
head = torch.load('head0.pt')
test(encoder, head, testdata, 'affect', criterion=torch.nn.L1Loss(),
     task="posneg-classification", modalnum=0, no_robust=True)

Testing:
loss: tensor(1.6484, device='cuda:0')
acc: 0.5350609756097561, 0.5291545189504373
Inference Time: 0.41711854934692383
Inference Params: 1454825


In [ ]:
encoder = GRU(74, 600, dropout=True, has_padding=False,
              batch_first=True, last_only=True).cuda()
head = MLP(600, 512, 1).cuda()

train(encoder, head, traindata, validdata, 200, task="regression", optimtype=torch.optim.AdamW, lr=2e-3,
      weight_decay=0.01, criterion=torch.nn.L1Loss(), save_encoder='encoder1.pt', save_head='head1.pt', modalnum= 1)


print("Testing:")
encoder = torch.load('encoder1.pt').cuda()
head = torch.load('head1.pt')
test(encoder, head, testdata, 'affect', criterion=torch.nn.L1Loss(),
     task="posneg-classification", modalnum=1, no_robust=True)

Epoch 0 train loss: tensor(1.3363, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 0 valid loss: tensor(1.3793, device='cuda:0')
Saving Best
Epoch 1 train loss: tensor(1.3369, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 1 valid loss: tensor(1.3824, device='cuda:0')
Epoch 2 train loss: tensor(1.3228, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2 valid loss: tensor(1.3863, device='cuda:0')
Epoch 3 train loss: tensor(1.3224, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 3 valid loss: tensor(1.3865, device='cuda:0')
Epoch 4 train loss: tensor(1.3239, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 4 valid loss: tensor(1.3818, device='cuda:0')
Epoch 5 train loss: tensor(1.3253, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 5 valid loss: tensor(1.3865, device='cuda:0')
Epoch 6 train loss: tensor(1.3231, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 6 valid loss: tensor(1.4417, device='cuda:0')
Epoch 7 train loss: tensor(1.3386, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 7 valid lo

In [ ]:
encoder = GRU(300, 600, dropout=True, has_padding=False,
              batch_first=True, last_only=True).cuda()
head = MLP(600, 512, 1).cuda()

train(encoder, head, traindata, validdata, 200, task="regression", optimtype=torch.optim.AdamW, lr=2e-3,
      weight_decay=0.01, criterion=torch.nn.L1Loss(), save_encoder='encoder2.pt', save_head='head2.pt', modalnum= 2)


print("Testing:")
encoder = torch.load('encoder2.pt').cuda()
head = torch.load('head2.pt')
test(encoder, head, testdata, 'affect', criterion=torch.nn.L1Loss(),
     task="posneg-classification", modalnum=2, no_robust=True)

Epoch 0 train loss: tensor(1.4751, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 0 valid loss: tensor(1.5246, device='cuda:0')
Saving Best
Epoch 1 train loss: tensor(1.5094, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 1 valid loss: tensor(1.3872, device='cuda:0')
Saving Best
Epoch 2 train loss: tensor(1.3406, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2 valid loss: tensor(1.3727, device='cuda:0')
Saving Best
Epoch 3 train loss: tensor(1.3223, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 3 valid loss: tensor(1.3896, device='cuda:0')
Epoch 4 train loss: tensor(1.3222, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 4 valid loss: tensor(1.3694, device='cuda:0')
Saving Best
Epoch 5 train loss: tensor(1.3215, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 5 valid loss: tensor(1.3795, device='cuda:0')
Epoch 6 train loss: tensor(1.3204, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 6 valid loss: tensor(1.3787, device='cuda:0')
Epoch 7 train loss: tensor(1.3184, device='cuda:0', grad

In [ ]:
pwd

'/content/MultiBench'

In [ ]:
import torch
import sys
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
# sys.path.append(os.getcwd())
# sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from training_structures.Supervised_Learning import train, test # noqa
from fusions.mult import MULTModel # noqa
from unimodals.common_models import Identity, MLP # noqa
from datasets.affect.get_data import get_dataloader # noqa
from fusions.common_fusions import Concat, MultiplicativeInteractions2Modal, MultiplicativeInteractions3Modal # noqa
traindata, validdata, test_robust = get_dataloader('/content/MultiBench/mosi_raw.pkl', robust_test=False, max_pad=True)


class HParams():
        num_heads = 8
        layers = 4
        attn_dropout = 0.1
        attn_dropout_modalities = [0,0,0.1]
        relu_dropout = 0.1
        res_dropout = 0.1
        out_dropout = 0.1
        embed_dropout = 0.2
        embed_dim = 40
        attn_mask = True
        output_dim = 1
        all_steps = False

encoders = [Identity().cuda(), Identity().cuda(), Identity().cuda()]
# encoders = [Identity().cuda(), Identity().cuda()]
# encoders = [GRU(35, 600, dropout=True, has_padding=False,
#               batch_first=True, last_only=True).cuda(),
#             GRU(74, 600, dropout=True, has_padding=False,
#               batch_first=True, last_only=True).cuda(),
#             GRU(300, 600, dropout=True, has_padding=False,
#               batch_first=True, last_only=True).cuda()]
fusion = MULTModel(3, [47, 74, 300], hyp_params=HParams).cuda()
# fusion = MultiplicativeInteractions3Modal(input_dims= [47, 74, 300], output_dim= 1, task= "affect").cuda()
# fusion = MultiplicativeInteractions3Modal(input_dims= [47, 74, 300], output_dim= 1, output= "vector").cuda()
# fusion = MULTModel(3, [371, 81, 300], hyp_params=HParams).cuda()
head = Identity().cuda()
# head = MLP(600, 512, 1).cuda()
train(encoders, fusion, head, traindata, validdata, 200, task="regression", optimtype=torch.optim.AdamW, early_stop=False, is_packed=False, lr=1e-3, clip_val=1.0, save='mosi_mult_best.pt', weight_decay=0.01, objective=torch.nn.L1Loss())

print("Testing:")
model = torch.load('mosi_mult_best.pt').cuda()

test(model=model, test_dataloaders_all=test_robust, dataset='mosi', is_packed=False,
     criterion=torch.nn.L1Loss(), task='posneg-classification', no_robust=True)

/content/MultiBench/fusions/common_fusions.py:141: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.W)
/content/MultiBench/fusions/common_fusions.py:144: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.U)
/content/MultiBench/fusions/common_fusions.py:147: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.V)
/content/MultiBench/fusions/common_fusions.py:149: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.b)
/content/MultiBench/fusions/common_fusions.py:155: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.W)
/content/MultiBench/fusions/common_fusions.py:157: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

dict_keys(['vision', 'audio', 'text', 'labels', 'id'])

In [ ]:
traindata.dataset.dataset['vision'].shape

(1283, 50, 35)

In [ ]:
traindata.dataset.dataset['audio'].shape

(1283, 50, 74)

In [ ]:
traindata.dataset.dataset['text'].shape

(1283, 50, 300)

In [ ]:
import torch
import sys
import os

sys.path.append(os.getcwd())
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from private_test_scripts.all_in_one import all_in_one_train  # noqa
from training_structures.Supervised_Learning import train, test  # noqa
from unimodals.common_models import GRUWithLinear, MLP  # noqa
from datasets.affect.get_data import get_dataloader  # noqa
from fusions.common_fusions import Concat, TensorFusion  # noqa


# mosi_data.pkl, mosei_senti_data.pkl
# mosi_raw.pkl, mosei_raw.pkl, sarcasm.pkl, humor.pkl
# raw_path: mosi.hdf5, mosei.hdf5, sarcasm_raw_text.pkl, humor_raw_text.pkl
traindata, validdata, test_robust = get_dataloader(
    '/content/MultiBench/mosi_raw.pkl', robust_test=False)

# mosi/mosei
encoders = [GRUWithLinear(35, 64, 4, dropout=True, has_padding=True).cuda(),
            GRUWithLinear(74, 128, 19, dropout=True, has_padding=True).cuda(),
            GRUWithLinear(300, 512, 79, dropout=True, has_padding=True).cuda()]
head = MLP(8000, 512, 1).cuda()

# humor/sarcasm
# encoders=[GRUWithLinear(371,512,4,dropout=True,has_padding=True).cuda(), \
#     GRUWithLinear(81,256,19,dropout=True,has_padding=True).cuda(),\
#     GRUWithLinear(300,600,79,dropout=True,has_padding=True).cuda()]
# head=MLP(8000,512,1).cuda()

fusion = TensorFusion().cuda()

train(encoders, fusion, head, traindata, validdata, 200, task="regression", optimtype=torch.optim.AdamW,
      early_stop=False, is_packed=True, lr=1e-3, save='mosi_tf_best.pt', weight_decay=0.01, objective=torch.nn.L1Loss())

print("Testing:")
model = torch.load('mosi_tf_best.pt').cuda()

test(model=model, test_dataloaders_all=test_robust, dataset='mosi',
     is_packed=True, criterion=torch.nn.L1Loss(), task='posneg-classification', no_robust=True)

Epoch 0 train loss: tensor(1.1820, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 0 valid loss: 1.2453186511993408
Saving Best
Epoch 1 train loss: tensor(1.0084, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 1 valid loss: 1.1885693073272705
Saving Best
Epoch 2 train loss: tensor(0.9224, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2 valid loss: 1.1282635927200317
Saving Best
Epoch 3 train loss: tensor(0.8060, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 3 valid loss: 1.0638257265090942
Saving Best
Epoch 4 train loss: tensor(0.8006, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 4 valid loss: 1.0648655891418457
Epoch 5 train loss: tensor(0.7058, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 5 valid loss: 1.0146338939666748
Saving Best
Epoch 6 train loss: tensor(0.6695, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 6 valid loss: 0.9935763478279114
Saving Best
Epoch 7 train loss: tensor(0.6320, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 7 valid loss: 0.9779022336006165
Saving B